In [2]:
#data processing
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1725/1725 [00:00<00:00, 10449.11 examples/s]


In [3]:
#postprocessing to our tokenized_datasets
#1- Remove the columns corresponding to values the model does not expect (like the sentence1 and sentence2 columns).
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
#2- Rename the column label to labels (because the model expects the argument to be named labels).
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#3- Set the format of the datasets so they return PyTorch tensors instead of lists.
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [4]:
#define dataloader:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [5]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78])}

finished data preprocessing

In [6]:
# instantiate model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

outputs = model(**batch)  #we pass just one batch to the model
print(outputs.loss, outputs.logits.shape)  #batch_size = 8

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.6786, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [7]:
#optimizer and scheduler
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


/home/qxy699/hugging_face/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
#The training loop
# we will want to use the GPU
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device


device(type='cuda')

In [9]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 1377/1377 [00:48<00:00, 28.28it/s]

100%|██████████| 1377/1377 [01:00<00:00, 28.28it/s]

In [10]:
#the evaluation loop
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8529411764705882, 'f1': 0.8972602739726028}

In [12]:
#----------------- Try it out-------------------------
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("stanfordnlp/sst2")
checkpoint = "bert-base-uncased"
checkpoint = AutoTokenizer.from_pretrained(checkpoint)



Generating test split: 100%|██████████| 1821/1821 [00:00<00:00, 501696.50 examples/s]


In [17]:
raw_datasets['train']

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 67349
})

In [21]:
def tokenize_function_SS2(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_datasets_sst = raw_datasets.map(tokenize_function_SS2, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1821/1821 [00:00<00:00, 47069.22 examples/s]


In [23]:
tokenized_datasets_sst = tokenized_datasets_sst.remove_columns(['idx', 'sentence'])
tokenized_datasets_sst = tokenized_datasets_sst.rename_column("label", "labels")
tokenized_datasets_sst.set_format("torch")
tokenized_datasets_sst.column_names

{'train': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'validation': ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
 'test': ['labels', 'input_ids', 'token_type_ids', 'attention_mask']}

In [24]:
train_dataloader_sst = DataLoader(tokenized_datasets_sst["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader_sst = DataLoader(tokenized_datasets_sst["validation"], batch_size=8, collate_fn=data_collator)


In [25]:
outputs_sst = model(**batch)
print(outputs_sst.loss, outputs_sst.logits.shape)

tensor(0.2315, device='cuda:0', grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [26]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader_sst:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 1377/1377 [27:17<00:00,  1.19s/it]


In [27]:
#the evaluation loop
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader_sst:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.48853211009174313, 'f1': 0.004464285714285714}